In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [6]:
# Load Sonar dataset

df = pd.read_csv(r"D:\data science\Assignments\assign\sonardataset.csv")

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Encode labels: M=1, R=0
y = np.where(y == 'M', 1, 0)

print("Dataset shape:", X.shape)


Dataset shape: (208, 60)


In [7]:
# Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [8]:
# Function to Evaluate Neural Network

def evaluate_mlp(hidden_units):
    kf = KFold(n_splits=13, shuffle=True, random_state=42)

    train_acc = []
    test_acc = []

    for train_idx, test_idx in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = MLPClassifier(
            hidden_layer_sizes=hidden_units,
            activation='logistic',
            solver='sgd',
            learning_rate_init=0.2,   # approx to paper
            momentum=0.0,
            max_iter=300,
            random_state=42
        )

        model.fit(X_train, y_train)

        train_acc.append(accuracy_score(y_train, model.predict(X_train)))
        test_acc.append(accuracy_score(y_test, model.predict(X_test)))

    return np.mean(train_acc)*100, np.std(train_acc)*100, \
           np.mean(test_acc)*100, np.std(test_acc)*100


In [9]:

# Run Experiments (Aspect-Independent)
hidden_layers = [(), (2,), (3,), (6,), (12,), (24,)]

results = []

for h in hidden_layers:
    tr_mean, tr_std, te_mean, te_std = evaluate_mlp(h)
    results.append([h if h else 0, tr_mean, tr_std, te_mean, te_std])

results_df = pd.DataFrame(
    results,
    columns=["Hidden Units", "Train %", "Train Std", "Test %", "Test Std"]
)

results_df

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\neural_network\_

,Hidden Units,Train %,Train Std,Test %,Test Std
0,0,90.705128,1.018290,75.000000,10.107594
1,"(2,)",89.142628,1.058481,76.923077,8.271467
2,"(3,)",90.705128,1.038580,76.442308,10.153227
3,"(6,)",90.705128,0.838480,77.403846,9.663342
4,"(12,)",89.623397,0.776872,79.326923,11.068139
5,"(24,)",91.105769,0.660754,77.884615,9.992601


In [10]:
# Balanced Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y,
    test_size=0.5,
    stratify=y,
    random_state=42
)

In [11]:
# Train Models with Different Hidden Units
dependent_results = []

for h in hidden_layers:
    model = MLPClassifier(
        hidden_layer_sizes=h,
        activation='logistic',
        solver='sgd',
        learning_rate_init=0.2,
        momentum=0.0,
        max_iter=300,
        random_state=42
    )

    model.fit(X_train, y_train)

    tr_acc = accuracy_score(y_train, model.predict(X_train)) * 100
    te_acc = accuracy_score(y_test, model.predict(X_test)) * 100

    dependent_results.append([h if h else 0, tr_acc, te_acc])

pd.DataFrame(
    dependent_results,
    columns=["Hidden Units", "Train Accuracy %", "Test Accuracy %"]
)

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\neural_network\_

,Hidden Units,Train Accuracy %,Test Accuracy %
0,0,96.153846,77.884615
1,"(2,)",88.461538,76.923077
2,"(3,)",92.307692,82.692308
3,"(6,)",94.230769,82.692308
4,"(12,)",93.269231,81.730769
5,"(24,)",94.230769,80.769231


In [12]:
# Final Evaluation
best_model = MLPClassifier(
    hidden_layer_sizes=(12,),
    activation='logistic',
    solver='sgd',
    learning_rate_init=0.2,
    momentum=0.0,
    max_iter=300,
    random_state=42
)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred)*100)
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 81.73076923076923

Confusion Matrix:
 [[40  9]
 [10 45]]

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.82      0.81        49
           1       0.83      0.82      0.83        55

    accuracy                           0.82       104
   macro avg       0.82      0.82      0.82       104
weighted avg       0.82      0.82      0.82       104



C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
